### Adjust Word Group Repeat With Youtube Link

In [142]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [143]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [144]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 40000  # 28 native word end index

# file extention
file_ext = "Sevengram"

In [145]:
def df_col_value_join_comma(df, df_columns_list):
    '''
    df_col_value_join_comma(df_test, ["video_id","start_time","end_time"])\n
    function used for selected column value join with comma in one row
    '''
    column_value_list = []
    for column in df_columns_list:
        list_var = df[f"{column}"].to_list()
        list_var_string = [str(x) for x in list_var] 
        list_var_join = ",".join(list_var_string)
        column_value_list.append(list_var_join)

    df_result = pd.DataFrame([[column_value_list[0],column_value_list[1],column_value_list[2]]], columns=df_columns_list)

    return df_result

In [146]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Talk Time 6/Result/3-Adjust Word Group Repeat With Youtube Link"

Path(path).mkdir(parents=True, exist_ok=True)

##### Link Count

In [147]:
path_folder = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Talk Time 6/Result/2-Find Word Group Youtube Link"
file = f"{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Youtube_Link_Manuel.xlsx"   ########*****
sheet = "Sheet1"  # Sheet1

In [148]:
df_link_selected_manuel = pd.read_excel(f"{path_folder}/{file}", sheet_name=f"{sheet}")
df_link_selected_manuel

,video_id,start_time,end_time,text
0,V9n8LpTPcfM,1002,1005,söylemek istediğin başka bir şey var mı
1,8ek8uhoEQ9w,662,664,söylemek istediğin başka bir şey var mı
2,hM5s4liNn7g,829,833,söylemek istediğin başka bir şey var mı
3,miIgOGcO2tY,649,651,söylemek istediğin başka bir şey var mı
4,ao2kpL29HwI,454,457,söylemek istediğin başka bir şey var mı
5,kQkG6D4c_xU,1076,1079,söylemek istediğin başka bir şey var mı
6,Odbd1stkcRk,6456,6459,bir iyi bir de kötü haberim var
7,O3Xde6lnWqw,349,352,bir iyi bir de kötü haberim var


In [149]:
df_link_selected_manuel["video_url"] = "https://www.youtube.com/watch?v="+df_link_selected_manuel['video_id'].map(str)+"&t="+df_link_selected_manuel['start_time'].map(str)+"s"
df_link_selected_manuel

,video_id,start_time,end_time,text,video_url
0,V9n8LpTPcfM,1002,1005,söylemek istediğin başka bir şey var mı,https://www.youtube.com/watch?v=V9n8LpTPcfM&t=...
1,8ek8uhoEQ9w,662,664,söylemek istediğin başka bir şey var mı,https://www.youtube.com/watch?v=8ek8uhoEQ9w&t=...
2,hM5s4liNn7g,829,833,söylemek istediğin başka bir şey var mı,https://www.youtube.com/watch?v=hM5s4liNn7g&t=...
3,miIgOGcO2tY,649,651,söylemek istediğin başka bir şey var mı,https://www.youtube.com/watch?v=miIgOGcO2tY&t=...
4,ao2kpL29HwI,454,457,söylemek istediğin başka bir şey var mı,https://www.youtube.com/watch?v=ao2kpL29HwI&t=...
5,kQkG6D4c_xU,1076,1079,söylemek istediğin başka bir şey var mı,https://www.youtube.com/watch?v=kQkG6D4c_xU&t=...
6,Odbd1stkcRk,6456,6459,bir iyi bir de kötü haberim var,https://www.youtube.com/watch?v=Odbd1stkcRk&t=...
7,O3Xde6lnWqw,349,352,bir iyi bir de kötü haberim var,https://www.youtube.com/watch?v=O3Xde6lnWqw&t=...


In [150]:
df_count = df_link_selected_manuel.groupby(["text"])[["text"]].count()
df_count.rename(columns={"text":"count"}, inplace=True)
df_count.sort_values(by="count", ascending=False, inplace=True)
df_count.reset_index(inplace=True)
df_count

,text,count
0,söylemek istediğin başka bir şey var mı,6
1,bir iyi bir de kötü haberim var,2


In [151]:
df_count = df_count[df_count["count"] > 2]

In [152]:
df_count_merge = pd.merge(df_link_selected_manuel, df_count, how="inner",on= "text")
df_count_merge.sort_values(by=["count","text"], ascending=False, inplace=True)
df_count_merge.reset_index(drop=True, inplace=True)
df_count_merge

,video_id,start_time,end_time,text,video_url,count
0,V9n8LpTPcfM,1002,1005,söylemek istediğin başka bir şey var mı,https://www.youtube.com/watch?v=V9n8LpTPcfM&t=...,6
1,8ek8uhoEQ9w,662,664,söylemek istediğin başka bir şey var mı,https://www.youtube.com/watch?v=8ek8uhoEQ9w&t=...,6
2,hM5s4liNn7g,829,833,söylemek istediğin başka bir şey var mı,https://www.youtube.com/watch?v=hM5s4liNn7g&t=...,6
3,miIgOGcO2tY,649,651,söylemek istediğin başka bir şey var mı,https://www.youtube.com/watch?v=miIgOGcO2tY&t=...,6
4,ao2kpL29HwI,454,457,söylemek istediğin başka bir şey var mı,https://www.youtube.com/watch?v=ao2kpL29HwI&t=...,6
5,kQkG6D4c_xU,1076,1079,söylemek istediğin başka bir şey var mı,https://www.youtube.com/watch?v=kQkG6D4c_xU&t=...,6


In [153]:
df_count.to_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Talk Time 6/Data/Deployment/Talk Time 6 {file_ext} Video List.xlsx", index=False)

In [154]:
df_count_merge.to_excel(f"{path_folder}/{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Youtube_Link_Manuel_Count.xlsx", index=False)   ########*****

##### Link Repeat

In [155]:
path_folder_file2 = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Talk Time 6/Result/2-Find Word Group Youtube Link/{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Youtube_Link_Manuel_Count.xlsx"   ########*****
sheet2 = "Sheet1"  # Sheet1

In [156]:
df_word_group_time_loc = pd.read_excel(f"{path_folder_file2}", sheet_name=f"{sheet2}")
#df_word_group_time_loc = pd.read_excel(f"Twogram Threegram Youtube Link Selected_Manuel.xlsx")   
df_word_group_time_loc

,video_id,start_time,end_time,text,video_url,count
0,V9n8LpTPcfM,1002,1005,söylemek istediğin başka bir şey var mı,https://www.youtube.com/watch?v=V9n8LpTPcfM&t=...,6
1,8ek8uhoEQ9w,662,664,söylemek istediğin başka bir şey var mı,https://www.youtube.com/watch?v=8ek8uhoEQ9w&t=...,6
2,hM5s4liNn7g,829,833,söylemek istediğin başka bir şey var mı,https://www.youtube.com/watch?v=hM5s4liNn7g&t=...,6
3,miIgOGcO2tY,649,651,söylemek istediğin başka bir şey var mı,https://www.youtube.com/watch?v=miIgOGcO2tY&t=...,6
4,ao2kpL29HwI,454,457,söylemek istediğin başka bir şey var mı,https://www.youtube.com/watch?v=ao2kpL29HwI&t=...,6
5,kQkG6D4c_xU,1076,1079,söylemek istediğin başka bir şey var mı,https://www.youtube.com/watch?v=kQkG6D4c_xU&t=...,6


In [157]:
search_list = df_word_group_time_loc.loc[:,"text"].unique()
len(search_list)

1

In [158]:
# other option 
# mUf7VNqChac =>  black screen
# 0_CDMstFg7M => 10sn
# bj1JRuyYeco => 20sn
# cElhIDdGz7M => screensaver
default_video_id = "Q-8I-uMUMYA"
df_link_default = pd.DataFrame(data=[["repeat",5,7,"repeat_again",f"{default_video_id}",f"https://www.youtube.com/watch?v={default_video_id}&t=0s"]], columns=["text","start_time","end_time","sentence","video_id","video_url"])
df_link_default

,text,start_time,end_time,sentence,video_id,video_url
0,repeat,5,7,repeat_again,Q-8I-uMUMYA,https://www.youtube.com/watch?v=Q-8I-uMUMYA&t=0s


In [159]:
#(df_result_repeat["end_time"].sum()-df_result_repeat["start_time"].sum())/60

In [160]:
# Each word group result convert to join result in one row
df_all_join_result = pd.DataFrame()
seq_num = 0  # option
for word_group in search_list:
    df_result_repeat = pd.DataFrame()
    # for repeat 1
    df_word_group_search_repeat1 = df_word_group_time_loc[df_word_group_time_loc["text"] == word_group]
    # for repeat 2
    try:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["text"] == word_group].sample(1)
    except:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["text"] == word_group].tail(1)
    df_word_group_search_repeat2.reset_index(drop=True, inplace=True)   
    # repeat block
    for i in range(len(df_word_group_search_repeat1)):        
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat1.iloc[[i,]]], axis=0)

    try:
        df_link_default_var = df_link_default
        word_time_diff_var = df_word_group_search_repeat2.loc[0 ,"end_time"] - df_word_group_search_repeat2.loc[0 ,"start_time"]
        if word_time_diff_var < 4:
            word_time_diff_var2 = 3
        else:
            word_time_diff_var2 = word_time_diff_var+1.0

        df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + word_time_diff_var2
    except:
        pass

    for j in range(3):
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat2], axis=0)
        df_result_repeat = pd.concat([df_result_repeat,df_link_default_var], axis=0)
        df_result_repeat.reset_index(drop=True, inplace=True)

    df_result_repeat_var = df_col_value_join_comma(df_result_repeat, ["video_id","start_time","end_time"]) 
    df_result_repeat_var.loc[0,["text"]] = f"{word_group}"  # option for search string
    df_result_repeat_var.loc[0,["count"]] = df_result_repeat.loc[0,"count"] # option for count num     
    df_all_join_result = pd.concat([df_all_join_result,df_result_repeat_var], axis=0)

df_all_join_result.reset_index(drop=True, inplace=True)
df_all_join_result["web_format"] = f"[play_lms_videos video_id="+df_all_join_result["video_id"].map(str)+" start_\
time="+df_all_join_result["start_time"].map(str)+" end_time="+df_all_join_result["end_time"].map(str)+"]"
df_all_join_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Youtube_Talk_Time_Join_{file_ext}.xlsx", index=False)

In [161]:
df_all_join_result

,video_id,start_time,end_time,text,count,web_format
0,"V9n8LpTPcfM,8ek8uhoEQ9w,hM5s4liNn7g,miIgOGcO2t...","1002,662,829,649,454,1076,649,5,649,5,649,5","1005,664,833,651,457,1079,651,8,651,8,651,8",söylemek istediğin başka bir şey var mı,6.0,"[play_lms_videos video_id=V9n8LpTPcfM,8ek8uhoE..."


#### Copy Move And Delete

In [162]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word_Youtube_Talk_Time_Join_*.xlsx")
output_file

['Turkish_20000_Word_Youtube_Talk_Time_Join_Sevengram.xlsx']

In [163]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [164]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass